In [126]:
import random
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [127]:
#############################
#### employee management ####
#############################
num_of_managers = 2;
num_of_lawyers = 1;
num_of_workers = 80;

In [133]:
##########################
#### company dynamics ####
##########################
# initial conditions
total_profit0 = 0 # initial total profit ($)
profit0 = 0 # initial profit ($/day)

# how effective employees are at their jobs
manager_perf_min = 0.8
lawyer_perf_min = 0.1
worker_perf_min = 0.2
manager_perf_range = 0.2
lawyer_perf_range = 0.1
worker_perf_range = 0.4
manager_perf = manager_perf_min + manager_perf_range * random.random()
lawyer_perf = lawyer_perf_min + lawyer_perf_range * random.random()
worker_perf = worker_perf_min + worker_perf_range * random.random()

# inertia of the company (managers are inertance)
m = 1 / (num_of_managers * manager_perf)
# damping of the company (lawyers are damping)
c = num_of_lawyers * lawyer_perf
# stiffness of the company (workers are compliance)
k = 1 / (num_of_workers * worker_perf)

def compute_profit_speed(total_profit, profit, investment):
    profit_speed = (investment - c * profit + k * total_profit) / m
    return profit_speed

def simulate_company(total_profit, profit, profit_speed, dt):
    # semi-implicit euler integration
    profit = profit + profit_speed * dt
    total_profit = total_profit + profit * dt
    return total_profit, profit

def update_company(total_profit, profit, investment, dt):
    profit_speed = compute_profit_speed(total_profit, profit, investment)
    total_profit, profit = simulate_company(total_profit, profit, profit_speed, dt)
    return total_profit, profit, profit_speed

In [129]:
####################################
#### custom investment function ####
####################################
def invest_custom(t):
    #return impulse(t, dt)
    return 0

In [130]:
#######################################
#### standard investment functions ####
#######################################
def invest_none():
    return 0

def invest_impulse(t, dt, scale=1, a=0):
    eps = 1.5 * dt # ensure the impulse function returns a non-zero value for a single timestep only
    if max(t - a, 0) < eps:
        val = scale * 1 / dt # the area below the impulse function should give 1 for a unit impulse function
    else:
        val = 0
    return val    

def invest_step(t, scale=1, a=0): 
    if t > a:
        val = scale
    else:
        val = 0
    return val

def invest_ramp(t, scale=1, a=0):
    if t > a:
        val = scale * t
    else:
        val = 0
    return val

def acceleration(t, scale=1, a=0):
    if t > a:
        val = scale * t**2
    else:
        val = 0
    return val

In [303]:
import ipycanvas as ipc
import ipyevents as ipe
from ipywidgets import Image

from time import time, sleep
from threading import Thread

import numpy as np

In [322]:
class Game(Thread):
    def __init__(self, x0, dx0, company_fun, investment_fun, t0=0, fps=60, canvas_width=600, canvas_height=300):        
        self.dt = 1 / fps # simulation timestep
        self.x0 = x0
        self.dx0 = dx0
        self.company_fun = company_fun
        self.investment_fun = investment_fun
        
        self.t0 = t0
        self.is_running = True

        self.canvas_width = canvas_width
        self.canvas_height = canvas_height
        self.init_pos = np.array([self.canvas_width / 2, self.canvas_height * 3 / 4])
        self.canvas = ipc.MultiCanvas(3, width=self.canvas_width, height=self.canvas_height)
        self.event = ipe.Event(source=self.canvas, watched_events=['keydown'])
        self.event.on_dom_event(self.handle_events) 
        
        self.t_hist = None
        self.xdxd2x_hist = None
        self.investment_hist = None
      
        self.draw_background(self.canvas[0])
        super(Game, self).__init__()
        
    # main loop
    def run(self):
        fps = 0
        x = self.x0 
        dx = self.dx0
        t = self.t0
        frame_time = self.dt
        start_time = time() - self.t0
        while(self.is_running):
            tick = time()
            
            """
            investment = self.investment_fun(t, self.dt)
            x, dx, d2x = self.company_fun(x, dx, investment, self.dt)
            self.draw_plots(self.canvas[1],?)
            """
            self.draw_text(self.canvas[2], fps, t)
            
            t = tick - start_time 
            fps = 1 / frame_time
            """
            if self.t_hist is None:
                self.t_hist = t
                self.xdxd2x_hist = np.concatenate((x, dx, d2x))
                self.investment_hist = investment
            else:
                self.t_hist = np.append(self.t_hist, t)
                self.xdxd2x_hist = np.vstack((self.xdxd2x_hist, np.concatenate((x, dx, d2x))))
                self.investment_hist = np.append(self.investment_hist, investment)
            """
            if self.dt - (time() - tick) > 0:
                sleep(self.dt - (time() - tick))
            frame_time = time() - tick
            
    # handle events
    def handle_events(self, event):
        # change invest_fun
        pass
        """"
        if event['key'] == 'ArrowRight':
            self.th_desired = self.th_desired + 1 / 180 * np.pi
        if event['key'] == 'ArrowLeft':
            self.th_desired = self.th_desired - 1 / 180 * np.pi
        """"
      
    # stop game           
    def stop(self):
        self.is_running = False

    def draw_plots(self, canvas):
        pass
    
    def draw_background(self, canvas):
        # not sure if there is another way, Image.width etc are empty?!
        background = Image.from_file('sprites/background.png')
        background_scale = 0.4
        background_width = 409
        background_height = 300 
        office = Image.from_file('sprites/office_building.png')
        office_scale = 0.2
        office_width = 170
        office_height = 200
        with ipc.hold_canvas(canvas):
            canvas.clear()
            #canvas.save()
            canvas.scale(background_scale)
            for i in range(0, int(600 / 409)+1):
                canvas.draw_image(background, i * (background_width / background_scale), (self.canvas_height - background_height) / background_scale)
            canvas.scale(1 / background_scale)
            canvas.scale(office_scale)
            canvas.draw_image(office, (self.canvas_width / 2 - office_width / 2) / office_scale, (self.canvas_height - office_height) / office_scale)
            canvas.scale(1 / office_scale)

    # draw text
    def draw_text(self, canvas, fps, time):
        # convert radians to degrees
        def to_degrees(radians):
            return radians / np.pi * 180

        # convert degrees into -180 +180 degrees
        def to_180degrees(degrees):
            while degrees > 180:
                degrees = degrees - 360
            while degrees < -180:
                degrees = degrees + 360
            return degrees
        
        with ipc.hold_canvas(canvas):
            pos = (5, 20)
            spacing = 14
            canvas.clear()
            canvas.font = '12px serif'
            canvas.fill_text('FPS: {:.0f}'.format(fps), pos[0], pos[1])
            canvas.fill_text('Time: {:.1f} s'.format(time), pos[0], pos[1] + spacing)
            """"
            canvas.fill_text('\u03B8: {:.1f}\u00B0'.format(to_180degrees(to_degrees(y[0]))), pos[0], pos[1] + 2 * spacing)
            canvas.fill_text('\u03C6: {:.1f}\u00B0'.format(to_180degrees(to_degrees(y[1]))), pos[0], pos[1] + 3 * spacing)
            canvas.fill_text('pos: {:.1f} m'.format(y[1] * 2 * r), pos[0], pos[1] + 3 * spacing)
            canvas.fill_text('vel: {:.1f} m/s'.format(dy[1] * 2 * r), pos[0], pos[1] + 4 * spacing)
            canvas.fill_text('acc: {:.1f} m/s\u00B2'.format(d2y[1] * 2 * r), pos[0], pos[1] + 5 * spacing)
            pos = (265, 20)
            centering_shift = 75
            canvas.font = '14px serif'
            canvas.fill_text('Target \u03B8: {:.1f}\u00B0'.format(to_180degrees(to_degrees(th_desired))), pos[0], pos[1])
            canvas.fill_text('(Use left and right arrow keys to change.)', pos[0] - centering_shift, pos[1] + spacing)    
            """"

In [329]:
try: # try to stop the game first in order to avoid creating redundant threads
    game.stop()
except NameError:
    pass
game = Game(0, 0, [], [], t0=0, fps=60, canvas_width=800, canvas_height=300)
game.start()
display(game.canvas)

MultiCanvas(height=300, width=800)

In [330]:
game.stop()

In [ ]:
import plotly.express as px
samples = [None]  * 10000
for i in range(0, 10000):
    samples[i] = random.random()
    
fig = px.histogram(samples)
fig.show()